In [1]:
# import libraries
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import requests
import io

In [2]:
key_path =  r'C:\Users\XIAOJIN LI\OneDrive - The City University of New York\Documents\master 笔记\2022 fall\warehouse\cis-9440-361019-89a32eedd802.json'

In [3]:
# run this cell without changing anything to setup your credentials
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials,
                                 project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

bigquery client name is: <google.cloud.bigquery.client.Client object at 0x0000027B14EF1C10>
bigquery client data type is: <class 'google.cloud.bigquery.client.Client'>


In [4]:
dataset_id = 'cis-9440-361019.uber'   # PASTE THIS DATASET ID FROM ABOVE STEPS

dataset_id = dataset_id.replace(':', '.')
print(f"your dataset_id is: {dataset_id}")

your dataset_id is: cis-9440-361019.uber


In [5]:
data = pd.read_csv("uber_data.csv")

In [6]:
data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2016-03-01 06:17:10,2016-03-01 06:22:15,1,0.50,-73.990898,40.750519,1,N,-73.998245,40.750462,2,5.0,0.0,0.5,0.00,0.00,0.3,5.80
99996,1,2016-03-01 06:17:10,2016-03-01 06:32:41,1,3.40,-74.014488,40.718296,1,N,-73.982361,40.752529,1,14.0,0.0,0.5,2.00,0.00,0.3,16.80
99997,1,2016-03-01 06:17:10,2016-03-01 06:37:23,1,9.70,-73.963379,40.774097,1,N,-73.865028,40.770512,1,29.0,0.0,0.5,8.80,5.54,0.3,44.14
99998,2,2016-03-01 06:17:10,2016-03-01 06:22:09,1,0.92,-73.984901,40.763111,1,N,-73.970695,40.759148,1,5.5,0.5,0.5,1.36,0.00,0.3,8.16


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               100000 non-null  int64  
 1   tpep_pickup_datetime   100000 non-null  object 
 2   tpep_dropoff_datetime  100000 non-null  object 
 3   passenger_count        100000 non-null  int64  
 4   trip_distance          100000 non-null  float64
 5   pickup_longitude       100000 non-null  float64
 6   pickup_latitude        100000 non-null  float64
 7   RatecodeID             100000 non-null  int64  
 8   store_and_fwd_flag     100000 non-null  object 
 9   dropoff_longitude      100000 non-null  float64
 10  dropoff_latitude       100000 non-null  float64
 11  payment_type           100000 non-null  int64  
 12  fare_amount            100000 non-null  float64
 13  extra                  100000 non-null  float64
 14  mta_tax                100000 non-nul

In [8]:
data["tpep_pickup_datetime"] = pd.to_datetime(data['tpep_pickup_datetime'])

In [9]:
data["tpep_dropoff_datetime"] = pd.to_datetime(data['tpep_dropoff_datetime'])

In [10]:
data['trip_id'] = data.index

In [11]:
data.isnull().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
pickup_longitude         0
pickup_latitude          0
RatecodeID               0
store_and_fwd_flag       0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
trip_id                  0
dtype: int64

In [12]:
data = data.drop_duplicates().reset_index(drop=True)

In [13]:
print(f"number of duplicate rows: {len(data[data.duplicated()])}")

number of duplicate rows: 0


In [14]:
# create and run a function to ceate data profiling dataframe

def create_data_profiling_df(data):
    
    # create an empty dataframe to gather information about each column
    data_profiling_df = pd.DataFrame(columns = ["column_name",
                                                "column_type",
                                                "unique_values",
                                                "duplicate_values",
                                                "null_values",
                                                "non_null_values"])

    # loop through each column to add rows to the data_profiling_df dataframe
    for column in data.columns:

        info_dict = {}

        try:
            info_dict["column_name"] = column
            info_dict["column_type"] = data[column].dtypes
            info_dict["unique_values"] = len(data[column].unique())
            info_dict["duplicate_values"] = data[column].count() - len(data[column].dropna().unique())
            info_dict["null_values"] = data[column].isna().sum()
            info_dict["non_null_values"] = data[column].count()

        except:
            print(f"unable to read column: {column}, you may want to drop this column")

        data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)

    data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                                  ascending = [False, False],
                                  inplace=True)
    
    return data_profiling_df

In [15]:
# view your data profiling dataframe
data_profiling_df = create_data_profiling_df(data = data)
data_profiling_df

C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_10244\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_10244\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_10244\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_10244\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from 

,column_name,column_type,unique_values,duplicate_values,null_values,non_null_values
19,trip_id,int64,100000,0,0,100000
2,tpep_dropoff_datetime,datetime64[ns],39981,60019,0,100000
1,tpep_pickup_datetime,datetime64[ns],38332,61668,0,100000
10,dropoff_latitude,float64,28377,71623,0,100000
6,pickup_latitude,float64,25082,74918,0,100000
9,dropoff_longitude,float64,15850,84150,0,100000
5,pickup_longitude,float64,13057,86943,0,100000
4,trip_distance,float64,2303,97697,0,100000
18,total_amount,float64,2058,97942,0,100000
15,tip_amount,float64,1036,98964,0,100000


In [16]:
datetime_dim = data.copy()

In [17]:
datetime_dim = datetime_dim[["tpep_dropoff_datetime","tpep_pickup_datetime"]]
#DELETE DUP
datetime_dim = datetime_dim.drop_duplicates().reset_index(drop=True)

In [18]:
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
# Aad a ID for this dimension
datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
unique_row =['tpep_dropoff_datetime','tpep_pickup_datetime']
data = data.merge(datetime_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

In [20]:
datetime_dim

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:07:55,0,1,3,2016,1
1,1,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:11:06,0,1,3,2016,1
2,2,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:31:06,0,1,3,2016,1
3,3,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1
4,4,2016-03-01 00:00:01,0,1,3,2016,1,2016-03-01 00:16:04,0,1,3,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99848,99848,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:22:15,6,1,3,2016,1
99849,99849,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:32:41,6,1,3,2016,1
99850,99850,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:37:23,6,1,3,2016,1
99851,99851,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:22:09,6,1,3,2016,1


In [21]:
#pickup_location_dim
pickup_location_dim = data.copy()
pickup_location_dim = data[['pickup_longitude', 'pickup_latitude']]
pickup_location_dim = pickup_location_dim.drop_duplicates().reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 
unique_row =['pickup_longitude','pickup_latitude']
data = data.merge(pickup_location_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')
pickup_location_dim

,pickup_location_id,pickup_latitude,pickup_longitude
0,0,40.765152,-73.976746
1,1,40.767925,-73.983482
2,2,40.644810,-73.782021
3,3,40.769814,-73.863419
4,4,40.792183,-73.971741
...,...,...,...
98050,98050,40.750519,-73.990898
98051,98051,40.718296,-74.014488
98052,98052,40.774097,-73.963379
98053,98053,40.763111,-73.984901


In [22]:
#drop off dim
dropoff_location_dim = data.copy()
dropoff_location_dim = data[['dropoff_longitude', 'dropoff_latitude']]
dropoff_location_dim = dropoff_location_dim.drop_duplicates().reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]
unique_row =['dropoff_longitude', 'dropoff_latitude']
data = data.merge(dropoff_location_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')
dropoff_location_dim

,dropoff_location_id,dropoff_latitude,dropoff_longitude
0,0,40.746128,-74.004265
1,1,40.733166,-74.005943
2,2,40.675770,-73.974541
3,3,40.757767,-73.969650
4,4,40.695053,-74.177170
...,...,...,...
98412,98412,40.750462,-73.998245
98413,98413,40.752529,-73.982361
98414,98414,40.770512,-73.865028
98415,98415,40.759148,-73.970695


In [23]:
#trip_distance_dim
trip_distance_dim = data.copy()
trip_distance_dim = data[['trip_distance']]
trip_distance_dim = trip_distance_dim.drop_duplicates().reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]
unique_row =['trip_distance']
data = data.merge(trip_distance_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')
trip_distance_dim

,trip_distance_id,trip_distance
0,0,2.50
1,1,2.90
2,2,19.98
3,3,10.78
4,4,30.43
...,...,...
2298,2298,23.38
2299,2299,15.91
2300,2300,21.11
2301,2301,20.63


In [24]:
passenger_count_dim= data.copy()
passenger_count_dim = data[['passenger_count']]
passenger_count_dim = passenger_count_dim.drop_duplicates().reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]
unique_row =['passenger_count']
data = data.merge(passenger_count_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')
passenger_count_dim

,passenger_count_id,passenger_count
0,0,1
1,1,2
2,2,3
3,3,5
4,4,6
5,5,4
6,6,0


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

In [26]:
#rate_code_dim
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}
rate_code_dim= data.copy()
rate_code_dim = data[['RatecodeID']]
rate_code_dim = rate_code_dim.drop_duplicates().reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]
unique_row =['RatecodeID']
data = data.merge(rate_code_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')
rate_code_dim

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,3,Newark
2,2,2,JFK
3,3,5,Negotiated fare
4,4,4,Nassau or Westchester
5,5,6,Group ride


In [27]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim=data.copy()
payment_type_dim = data[['payment_type']]
payment_type_dim = payment_type_dim.drop_duplicates().reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]
unique_row =['payment_type']
data = data.merge(payment_type_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

payment_type_dim

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,2,Cash
2,2,3,No charge
3,3,4,Dispute


In [28]:
data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'trip_id', 'datetime_id',
       'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
       'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday',
       'pickup_location_id', 'dropoff_location_id', 'trip_distance_id',
       'passenger_count_id', 'rate_code_id', 'rate_code_name',
       'payment_type_id', 'payment_type_name'],
      dtype='object')

In [29]:
fact_table= data[['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
       'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
       'pickup_location_id', 'dropoff_location_id', 'payment_type_id',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount']]
fact_table 

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,0,1,0,N,1,1,0,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,1,2,0,N,2,2,0,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,2,3,0,N,3,3,0,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,3,3,4,1,N,4,4,0,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,99848,0,19,0,N,98050,98412,1,5.0,0.0,0.5,0.00,0.00,0.3,5.80
99996,99996,1,99849,0,71,0,N,98051,98413,0,14.0,0.0,0.5,2.00,0.00,0.3,16.80
99997,99997,1,99850,0,295,0,N,98052,98414,0,29.0,0.0,0.5,8.80,5.54,0.3,44.14
99998,99998,2,99851,0,152,0,N,98053,98415,0,5.5,0.5,0.5,1.36,0.00,0.3,8.16


In [30]:
'''
another way to merge 
fact_table = data.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]
'''

"\nanother way to merge \nfact_table = data.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id')              .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id')              .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id')              .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id')              .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id')              .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id')              [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',\n               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',\n               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',\n               'improvement_surcharge', 'total_amount']]\n"

In [31]:
# create a function to load dataframes to BigQuery

def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id #change 301800 to match your project id

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"
    
    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)
        
    print(f"completed job {load_job}")

In [32]:
#Sending fact_table 

load_table_to_bigquery(df = fact_table,
                       table_name = "fact_table",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=268d4b7b-5d4e-44a9-8578-075ff1a2583f>


In [33]:
#Sending payment_type_dim 
load_table_to_bigquery(df = payment_type_dim,
                       table_name = "payment_type_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=3cb8e2ca-c52e-4c17-af35-b098d5ed4d9e>


In [34]:
#Sending rate_code_dim 
load_table_to_bigquery(df = rate_code_dim,
                       table_name = "rate_code_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=ba4257df-c6b7-4af5-bf9b-13f70f616fe8>


In [35]:
#Sending datetime_dim 
load_table_to_bigquery(df = datetime_dim,
                       table_name = "datetime_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=49d141c2-5157-46f4-95c3-4599dccc5589>


In [36]:
#Sending trip_distance_dim 
load_table_to_bigquery(df = trip_distance_dim,
                       table_name = "trip_distance_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=6b1119e6-885c-4e9c-833c-85c1d752163d>


In [37]:
#Sending dropoff_location_dim 
load_table_to_bigquery(df = dropoff_location_dim,
                       table_name = "dropoff_location_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=58f1b8c3-9a0f-43b0-8f5b-37a0dae5bf8e>


In [38]:
#Sending pickup_location_dim 
load_table_to_bigquery(df = pickup_location_dim,
                       table_name = "pickup_location_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=5914200f-49ae-4897-a201-ba9bc32cb92c>


In [39]:
#Sending passenger_count_dim 
load_table_to_bigquery(df = passenger_count_dim,
                       table_name = "passenger_count_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=99b94698-cb59-4c7d-8a6c-3e822ed8e885>
